# GSM1D-VEVPD

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown

from bmcs_matmod.api import (
    GSMModel, GSM1D_VEVPD, TimeFnMonotonicAscending
)

gsm = GSMModel(GSM1D_VEVPD)

display(Markdown(GSM1D_VEVPD.latex_potentials()))

## Monotonic strain-driven loading

In [ ]:
mparams = dict(E = 20, K = 0, S = 100, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=100, eta_ve=300)
gsm.set_params(**mparams)
eps_max = 10.0
final_time = 10
# mparams = dict(E = 20e+6, K = 0e+6, S = 0.000005, c = 1, r = 1, f_c = 5e+3, eta_vp=50+6, eta_ve=50e+6)
# final_time = 10
# eps_max = 1.5e-3 
n_t = 151
t_t = np.linspace(0, final_time, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
F_response = gsm.get_F_response(eps_n_t, t_t)


In [ ]:
gsm.gsm_def.Eps_vars

In [ ]:
gsm.gsm_def

In [ ]:
rd_ec = F_response
# Plot viscoplastic strain and damage along time
iter_t = rd_ec.iter_t
lam_t = rd_ec.lam_t
eps_t = rd_ec.eps_t[:, 0]
sig_t = rd_ec.sig_t[:, 0, 0]
eps_v_t = rd_ec.Eps_t.eps_v_a[:, 0, 0]
eps_p_t = rd_ec.Eps_t.eps_p_a[:, 0, 0]
omega_t = rd_ec.Eps_t.omega_a[:, 0, 0]
z_t = rd_ec.Eps_t.z_a[:, 0, 0]
sig_v_t = rd_ec.Sig_t.sig_v_a[:, 0, 0]
sig_p_t = rd_ec.Sig_t.sig_p_a[:, 0, 0]
Y_t = rd_ec.Sig_t.Y_a[:, 0, 0]
Z_t = rd_ec.Sig_t.Z_a[:, 0, 0]

In [ ]:
sig_out_t = mparams['E'] * (eps_t - eps_v_t -  eps_p_t)

In [ ]:
fig, ((ax_sig, ax_omega),
    (ax_lam, ax_z),
    (ax_Y, ax_compare_sig),
    (ax_rate, ax_unused)) = plt.subplots(4, 2, figsize=(12, 16))
fig.canvas.header_visible = False

# First row
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(t_t, sig_p_t, color='cadetblue', label=r'$t$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$t$', color='cadetblue')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

# Second row
ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_t - eps_p_t, label=r'$\varepsilon - \varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.plot(t_t, eps_t - eps_p_t - eps_v_t, label=r'$\varepsilon - \varepsilon_\mathrm{vp} - \varepsilon_\mathrm{ve}$', 
        color='cadetblue', ls='dotted', lw=1)
ax_eps.fill_between(t_t, eps_t, eps_t - eps_p_t, color='red', alpha=0.3, label=r'$\varepsilon_\mathrm{vp}$')
ax_eps.fill_between(t_t, eps_t - eps_p_t, eps_t - eps_p_t - eps_v_t, color='blue', alpha=0.3, label='viscoelastic')
ax_eps.fill_between(t_t, eps_t - eps_p_t - eps_v_t, 0, color='green', alpha=0.3, label='elastic')
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Third row
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, :], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_t, label=r'$\varepsilon$', color='blue')
ax_z_eps_p.plot(t_t, eps_v_t, label=r'$\varepsilon_\mathrm{ve}$', color='blue', ls='dotted')
ax_z_eps_p.plot(t_t, eps_v_t + eps_p_t, label=r'$\varepsilon_\mathrm{ve} + \varepsilon_\mathrm{vp}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')
ax_z_eps_p.legend()

ax_Y.plot(t_t, Y_t, label=r'$Y$', color='magenta')
ax_Y.set_xlabel(r'$t$')
ax_Y.set_ylabel(r'$Y$', color='magenta')
ax_Y.legend()

ax_compare_sig.plot(t_t, sig_t, color='cadetblue', label=r'$\sigma$')
ax_compare_sig.plot(t_t, sig_p_t, color='cadetblue', ls='dashed', label=r'$\sigma_\mathrm{vp}$')
ax_compare_sig.plot(t_t, sig_v_t, color='cadetblue', ls='dotted', label=r'$\sigma_\mathrm{ve}$')
ax_compare_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_compare_sig.set_xlabel(r'$t$', color='cadetblue')
ax_compare_sig.plot(t_t, sig_out_t, color='red', label=r'$\sigma$')
ax_compare_sig.legend()

# Fourth row: rates of strains
dt = t_t[1:] - t_t[:-1]
eps_rate = np.diff(eps_t) / dt
eps_v_rate = np.diff(eps_v_t) / dt
eps_p_rate = np.diff(eps_p_t) / dt
t_mid = t_t[:-1] + dt / 2

ax_rate.plot(t_mid, eps_rate, label='Total strain rate', color='black')
ax_rate.plot(t_mid, eps_v_rate, label='Viscoelastic rate', color='blue', ls='dotted')
ax_rate.plot(t_mid, eps_p_rate, label='Plastic rate', color='red', ls='dashed')
ax_rate.set_xlabel(r'$t$')
ax_rate.set_ylabel('Strain rate')
ax_rate.legend()

# Hide unused axis in the bottom-right subplot
ax_unused.axis('off')

plt.tight_layout()
plt.show()

## Cyclic strain-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
eps_max = 2
n_t_ec_cyclic = 4 * n_t
f_time = 30
t_t = np.linspace(0, f_time, n_t_ec_cyclic)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*f_time, 0.5*f_time, 0.75*f_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
final_time = 1
eps_max = 2
t_t = np.linspace(0, final_time, 3000)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*final_time, 0.5*final_time, 0.75*final_time)
t_t = np.hstack([t_t, t_t[-1] + t_t[1:]])
eps_n_t = np.hstack([eps_n_t, -eps_n_t[1:]])
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
mparams = dict(E = 20, K = 5, S =200, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=1e-5, eta_ve=10)
gsm.set_params(**mparams)
rc_ed = gsm.get_F_response(eps_n_t, t_t)
# Plot viscoplastic strain and damage along time
t_t = rd_ec.t_t
iter_t = rd_ec.iter_t
lam_t = rd_ec.lam_t
eps_t = rd_ec.eps_t[:, 0]
sig_t = rd_ec.sig_t[:, 0, 0]
eps_v_t = rd_ec.Eps_t.eps_v_a[:, 0, 0]
eps_p_t = rd_ec.Eps_t.eps_p_a[:, 0, 0]
omega_t = rd_ec.Eps_t.omega_a[:, 0, 0]
z_t = rd_ec.Eps_t.z_a[:, 0, 0]
sig_v_t = rd_ec.Sig_t.sig_v_a[:, 0, 0]
sig_p_t = rd_ec.Sig_t.sig_p_a[:, 0, 0]
Y_t = rd_ec.Sig_t.Y_a[:, 0, 0]
Z_t = rd_ec.Sig_t.Z_a[:, 0, 0]

In [ ]:
rd_ec.t_t.shape, t_t.shape, rd_ec.eps_t.shape, rd_ec.Sig_t.sig_p_a.shape, sig_p_t.shape

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))
fig.canvas.header_visible = False

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

# ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
# ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
# ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
# ax_sig.set_xlabel(r'$\varepsilon$')
# ax_sig.legend()

ax_sig.plot(t_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_iter.plot(t_t, eps_t, color='cadetblue', ls='dashed', label=r'$\varepsilon$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

# ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
# ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, :], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_t, label=r'$\varepsilon$', color='blue')
ax_z_eps_p.plot(t_t, eps_v_t, label=r'$\varepsilon_\mathrm{ve}$', color='blue', ls='dotted')
ax_z_eps_p.plot(t_t, eps_v_t + eps_p_t, label=r'$\varepsilon_\mathrm{ve} + \varepsilon_\mathrm{vp}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')
ax_z_eps_p.legend()

plt.tight_layout()
plt.show()